In [1]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [2]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("disable-gpu")

## Get xy coordinate directly from embeded navermap

In [635]:
chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)
url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1001&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=2'
# url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=2500&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'
driver.get(url) 

time.sleep(10)

source = driver.page_source 
data = bs(source, 'html.parser')

In [636]:
cont = data.findAll("div", {"class":"map_list"})[0]
ul = cont.ul.select("li[onclick]")

In [641]:
def get_siteinfo(bsobj):
    cont = bsobj.findAll("div", {"class":"map_list"})[0]
    ul = cont.ul.select("li[onclick]")
    cols = ['name','addr', 'call_num','coord_x','coord_y']
    df = pd.DataFrame(columns=cols)

    for i in range(len(ul)):
        camp = ul[i]
        xy = camp['onclick']
        x = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[1]
        y = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[0]
        name = camp.h2.text.split(']')[1].strip()
        addr = camp.find("li", {"class":"addr"}).get_text()
        call_num = camp.find("li", {"class":"call_num"}).get_text() if camp.find("li", {"class":"call_num"}) else ""
        val_list = [name, addr, call_num, x, y]
        result = dict(zip(cols, val_list))
        df = df.append(result, ignore_index=True)
    print(df.head(2))
    return df

In [719]:
cols = ['name','addr', 'call_num','coord_x','coord_y']
df_fn = pd.DataFrame(columns=cols)
limit_ = 1000

chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)

for i in range(1,4):
#     print(i)
    url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit={limit}&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex={page}'.format(limit=limit_, page=i)
    print(url)
    driver.get(url) 
    time.sleep(3)
    source = driver.page_source 
    data = bs(source, 'html.parser')
    
    df_rslt = get_siteinfo(data)
#     print(df_rslt.head())
    df_fn = df_fn.append(df_rslt)
    

https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1
       name                       addr       call_num      coord_x     coord_y
0     트리캠핑장  인천광역시 옹진군 선재로306번길 27-55                  126.5149936  37.2568236
1  마노아워터캠핑장          경남 합천군 봉산면 압곡3길 9  010-8667-6051  128.0578454  35.6424478
https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=2
         name                      addr       call_num      coord_x  \
0  평화의댐 오토캠핑장  강원 화천군 화천읍 비수구미길 145-145  070-7727-1290  127.8501067   
1      펫 트리파크     충북 충주시 앙성면 둔터로 625-31                 127.7103409   

      coord_y  
0  38.2065288  
1  37.1029014  
https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=3
          name                      addr       call_num      coord_x  \
0  제이드가든 오토캠핑장        강원도 춘천시 

In [720]:
df = df_fn.reset_index(drop=True)

In [669]:
df_fn.to_csv("../file/campingsite.csv")

* Najost Camp > 나조스트캠핑장
* (.*) > ''
* & > 앤
* 춘천Believing(빌리빙)캠핑장 > 춘천빌리빙캠핑장
* iFA캠프운악 > 캠프운악
* Black tree > 블랙트리캠핑장
* 영천 구룡산 오토&펜션 > 영천구룡산오토캠핑장
* (주)자연에너지-드림랜드오토캠핑장 > 드림랜드오토캠핑장
* 소회산리멍우리협곡관광농원(캠핑장) > 멍우리협곡캠핑장
* 변산반도국립공원 고사포 야영장 (임시) > 고사포야영장
* Camp 1950 by 민들레울 > camp1950

In [748]:
def name_for_tag(x):
    if x=="Najost Camp": return "나조스트캠핑장"
    elif x=="iFA캠프운악": return "캠프운악"
    elif x=="Black tree": return "블랙트리캠핑장"
    elif x=="영천 구룡산 오토&펜션": return "영천구룡산오토캠핑장"
    elif x=="(주)자연에너지-드림랜드오토캠핑장": return "드림랜드오토캠핑장"
    elif x=="소회산리멍우리협곡관광농원(캠핑장)": return "멍우리협곡캠핑장"
    elif x=="춘천Believing(빌리빙)캠핑장": return "춘천빌리빙캠핑장"
    elif x=="변산반도국립공원 고사포 야영장 (임시)": return "고사포야영장"
    elif x=="A&J오토캠핑장": return "AJ오토캠핑장"
    elif "&" in x: return re.sub("&","앤", x)
    elif re.search("\(.*\)", x): return re.sub("\(.*\)","", x)
    else: return x
#     else: return x.replace(" ","").strip()
        
df["hashtag_test"] = df["name"].apply(name_for_tag)
df["hashtag_test"] = df["hashtag_test"].apply(name_for_tag)
df["hashtag"] = df["hashtag_test"].apply(lambda x: x.replace(" ","").strip())
df[df["name"]!=df['hashtag_test']]

,name,addr,call_num,coord_x,coord_y,hashtag,hashtag_test
109,얼음골한옥펜션&캠핑장,경남 밀양시 산내면 하양지안길 11-19,,128.9528490,35.5984747,얼음골한옥펜션앤캠핑장,얼음골한옥펜션앤캠핑장
119,진부(오대산)관광농원,강원 평창군 진부면 아차골길 34-15,010-6343-6516,128.5672147,37.5555278,진부관광농원,진부관광농원
135,소백산 신선펜션 & 캠핑,충북 단양군 가곡면 새밭로 474,010-4963-7169,128.4228607,36.9958373,소백산신선펜션앤캠핑,소백산 신선펜션 앤 캠핑
177,(주)아웃오브파크,강원도 춘천시 남면 가옹개길 52-9,1522-1861-,127.5112565,37.7278127,아웃오브파크,아웃오브파크
181,505펜션&캠핑(오공오),충남 공주시 사곡면 호계황골길 43,010-7330-2505,127.0207700,36.5017433,505펜션앤캠핑,505펜션앤캠핑
182,(주)디노담양힐링파크 지점,전남 담양군 봉산면 탄금길 9-26,061-383-5155,126.9609528,35.2714369,디노담양힐링파크지점,디노담양힐링파크 지점
193,지리산오토캠핑장 (호호당),전라남도 구례군 산동면 수락폭포로 170-5,061-781-9114,127.4514518,35.3308948,지리산오토캠핑장,지리산오토캠핑장
205,링크독애견파크 글램핑&오토캠핑장,경기도 가평군 설악면 신천리 263-1,010-5382-1943,127.489931,37.661018,링크독애견파크글램핑앤오토캠핑장,링크독애견파크 글램핑앤오토캠핑장
242,춘천Believing(빌리빙)캠핑장,강원도 춘천시 사북면 지암리,010-8747-6416,127.6278216,37.9767025,춘천빌리빙캠핑장,춘천빌리빙캠핑장
260,변산반도국립공원 고사포 야영장 (임시),전라북도 부안군 변산면 운산리 441-11,063-582-7835,126.5092132,35.6626201,고사포야영장,고사포야영장


In [660]:
len(df_fn['name'].unique())

2406

## geocoding example

In [ ]:
cols = list(df.columns) + ['level1','level2','x','y']
df_fn = pd.DataFrame(columns=cols)
apikey = '6829FE20-AE45-312E-9D0A-418C7E77BF47'

for i in range(len(df)):
    dict0 = dict(df.iloc[i])
#     print(dict0)

    address = df['addr'][i]
    URL = 'http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={addr}&refine=true&simple=false&format=json&type=road&key={key}'.format(key=apikey, addr=address)
    r = requests.get(URL)
    data = r.json()
    try: 
        dict1 = data['response']['refined']['structure']
        dict1 = {k: dict1[k] for k in dict1.keys() & {'level1','level2'}}
        dict2 = data['response']['result']['point']
        dict0.update(dict1)
        dict0.update(dict2)
    except KeyError:
        dict1 = {'level1':None, 'level2':None, 'x':None, 'y':None}
        dict0.update(dict1)
    
    df_tmp = pd.DataFrame(json_normalize(dict0), columns=dict0.keys())
    df_fn = df_fn.append(df_tmp)
    print(df_fn.tail(5))